# Intro to Apache Spark

* [Intro to Spark slides](https://github.com/databricks/tech-talks/blob/master/2020-04-29%20%7C%20Intro%20to%20Apache%20Spark/Intro%20to%20Spark.pdf)
* What is a Spark DataFrame?
  * Read in the [NYT data set](https://github.com/nytimes/covid-19-data) 
* How to perform a distributed count?
* Transformations vs. Actions
* Spark SQL

[Spark docs](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html)

In [2]:
%fs ls databricks-datasets/COVID/covid-19-data/

path,name,size
dbfs:/databricks-datasets/COVID/covid-19-data/.git/,.git/,0
dbfs:/databricks-datasets/COVID/covid-19-data/LICENSE,LICENSE,1289
dbfs:/databricks-datasets/COVID/covid-19-data/NYT-readme.md,NYT-readme.md,1748
dbfs:/databricks-datasets/COVID/covid-19-data/README.md,README.md,14686
dbfs:/databricks-datasets/COVID/covid-19-data/us-counties.csv,us-counties.csv,3512129
dbfs:/databricks-datasets/COVID/covid-19-data/us-states.csv,us-states.csv,91276
dbfs:/databricks-datasets/COVID/covid-19-data/us.csv,us.csv,1856


## How do we represent this data?

![Unified Engine](https://files.training.databricks.com/images/105/unified-engine.png)


####At first there were RDDs...
* **R**esilient: Fault-tolerant
* **D**istributed: Across multiple nodes
* **D**ataset: Collection of partitioned data

RDDs are immutable once created and keep track of their lineage to enable failure recovery.

####... and then there were DataFrames
* Higher-level APIs
* User friendly
* Optimizations and performance improvements

![RDD vs DataFrames](https://files.training.databricks.com/images/105/rdd-vs-dataframes.png)

###Create a DataFrame from the NYT COVID data

In [6]:
covid_df = spark.read.csv("databricks-datasets/COVID/covid-19-data/us-counties.csv")
covid_df.show()

+----------+-----------+----------+-----+-----+------+
 _c0| _c1| _c2| _c3| _c4| _c5|
+----------+-----------+----------+-----+-----+------+
 date| county| state| fips|cases|deaths|
2020-01-21| Snohomish|Washington|53061| 1| 0|
2020-01-22| Snohomish|Washington|53061| 1| 0|
2020-01-23| Snohomish|Washington|53061| 1| 0|
2020-01-24| Cook| Illinois|17031| 1| 0|
2020-01-24| Snohomish|Washington|53061| 1| 0|
2020-01-25| Orange|California|06059| 1| 0|
2020-01-25| Cook| Illinois|17031| 1| 0|
2020-01-25| Snohomish|Washington|53061| 1| 0|
2020-01-26| Maricopa| Arizona|04013| 1| 0|
2020-01-26|Los Angeles|California|06037| 1| 0|
2020-01-26| Orange|California|06059| 1| 0|
2020-01-26| Cook| Illinois|17031| 1| 0|
2020-01-26| Snohomish|Washington|53061| 1| 0|
2020-01-27| Maricopa| Arizona|04013| 1| 0|
2020-01-27|Los Angeles|California|06037| 1| 0|
2020-01-27| Orange|California|06059| 1| 0|
2020-01-27| Cook| Illinois|17031| 1| 0|
2020-01-27| Snohomish|Washington|53061| 1| 0|
2020-01-28| Maricopa| Arizona|04013| 1| 0|
+----------+-----------+----------+-----+-----+------+
only showing top 20 rows

Let's look at the [Spark docs](https://spark.apache.org/docs/latest/index.html) to see what options we have to pass into the csv reader.

In [8]:
covid_df = spark.read.csv("databricks-datasets/COVID/covid-19-data/us-counties.csv", header=True, inferSchema=True)
covid_df.show()

+-------------------+-----------+----------+-----+-----+------+
 date| county| state| fips|cases|deaths|
+-------------------+-----------+----------+-----+-----+------+
2020-01-21 00:00:00| Snohomish|Washington|53061| 1| 0|
2020-01-22 00:00:00| Snohomish|Washington|53061| 1| 0|
2020-01-23 00:00:00| Snohomish|Washington|53061| 1| 0|
2020-01-24 00:00:00| Cook| Illinois|17031| 1| 0|
2020-01-24 00:00:00| Snohomish|Washington|53061| 1| 0|
2020-01-25 00:00:00| Orange|California| 6059| 1| 0|
2020-01-25 00:00:00| Cook| Illinois|17031| 1| 0|
2020-01-25 00:00:00| Snohomish|Washington|53061| 1| 0|
2020-01-26 00:00:00| Maricopa| Arizona| 4013| 1| 0|
2020-01-26 00:00:00|Los Angeles|California| 6037| 1| 0|
2020-01-26 00:00:00| Orange|California| 6059| 1| 0|
2020-01-26 00:00:00| Cook| Illinois|17031| 1| 0|
2020-01-26 00:00:00| Snohomish|Washington|53061| 1| 0|
2020-01-27 00:00:00| Maricopa| Arizona| 4013| 1| 0|
2020-01-27 00:00:00|Los Angeles|California| 6037| 1| 0|
2020-01-27 00:00:00| Orange|California| 6059| 1| 0|
2020-01-27 00:00:00| Cook| Illinois|17031| 1| 0|
2020-01-27 00:00:00| Snohomish|Washington|53061| 1| 0|
2020-01-28 00:00:00| Maricopa| Arizona| 4013| 1| 0|
2020-01-28 00:00:00|Los Angeles|California| 6037| 1| 0|
+-------------------+-----------+----------+-----+-----+------+
only showing top 20 rows

###How many records do we have?
* Instead of counting M&Ms, let's count the number of rows in the DataFrame

###What do we expect our Spark job to look like?
* How many stages?

In [10]:
covid_df.count()

Out[3]: 89772

### Let's write some Spark code!

* I want to look at only the information for the county I live in (Los Angeles)
* I want the most recent information at the top

In [12]:
(covid_df
 .sort(covid_df["date"].desc()) 
 .filter(covid_df["county"] == "Los Angeles")) 

Out[4]: DataFrame[date: timestamp, county: string, state: string, fips: int, cases: int, deaths: int]

**...nothing happened. Why?**

## Transformations vs Actions

There are two types of operations in Spark: transformations and actions.

Fundamental to Apache Spark are the notions that
* Transformations are **LAZY**
* Actions are **EAGER**

In [15]:
# same operations as above
(covid_df
 .sort(covid_df["date"].desc()) 
 .filter(covid_df["county"] == "Los Angeles")) 

Out[5]: DataFrame[date: timestamp, county: string, state: string, fips: int, cases: int, deaths: int]

Why isn't is showing me results? **Sort** and **filter** are `transformations`, which are lazily evaluated in Spark.

Laziness has a number of benefits
* Not forced to load all data in the first step
  * Technically impossible with **REALLY** large datasets.
* Easier to parallelize operations 
  * N different transformations can be processed on a single data element, on a single thread, on a single machine. 
* Most importantly, it allows the framework to automatically apply various optimizations
  * This is also why we use Dataframes!
  
There's a lot Spark's **Catalyst** optimizer can do. Let's focus on only this situation. For more information, read [this blog!](https://databricks.com/blog/2015/04/13/deep-dive-into-spark-sqls-catalyst-optimizer.html)
  
![Catalyst](https://files.training.databricks.com/images/105/catalyst-diagram.png)

In [17]:
(covid_df
 .sort(covid_df["date"].desc()) 
 .filter(covid_df["county"] == "Los Angeles") 
 .show())  #action!

+-------------------+-----------+----------+----+-----+------+
 date| county| state|fips|cases|deaths|
+-------------------+-----------+----------+----+-----+------+
2020-04-25 00:00:00|Los Angeles|California|6037|19107| 895|
2020-04-24 00:00:00|Los Angeles|California|6037|18545| 850|
2020-04-23 00:00:00|Los Angeles|California|6037|17508| 797|
2020-04-22 00:00:00|Los Angeles|California|6037|16435| 729|
2020-04-21 00:00:00|Los Angeles|California|6037|15140| 663|
2020-04-20 00:00:00|Los Angeles|California|6037|13816| 617|
2020-04-19 00:00:00|Los Angeles|California|6037|12341| 600|
2020-04-18 00:00:00|Los Angeles|California|6037|12021| 576|
2020-04-17 00:00:00|Los Angeles|California|6037|11391| 495|
2020-04-16 00:00:00|Los Angeles|California|6037|10854| 455|
2020-04-15 00:00:00|Los Angeles|California|6037|10496| 402|
2020-04-14 00:00:00|Los Angeles|California|6037|10047| 360|
2020-04-13 00:00:00|Los Angeles|California|6037| 9420| 320|
2020-04-12 00:00:00|Los Angeles|California|6037| 9192| 296|
2020-04-11 00:00:00|Los Angeles|California|6037| 8873| 265|
2020-04-10 00:00:00|Los Angeles|California|6037| 8430| 241|
2020-04-09 00:00:00|Los Angeles|California|6037| 7955| 223|
2020-04-08 00:00:00|Los Angeles|California|6037| 7530| 198|
2020-04-07 00:00:00|Los Angeles|California|6037| 6910| 169|
2020-04-06 00:00:00|Los Angeles|California|6037| 6360| 147|
+-------------------+-----------+----------+----+-----+------+
only showing top 20 rows

###We can see the optimizations in action!
* Go to the Spark UI
* Click on the SQL query associated with your Spark job
* See the logical and physical plans!
  * The filter and sort have been swapped

## Spark SQL

In [20]:
covid_df.createOrReplaceTempView("covid")

In [21]:
%sql

SELECT * 
FROM covid

-- keys = date, grouping = county, values = cases

date,county,state,fips,cases,deaths
2020-01-21T00:00:00.000+0000,Snohomish,Washington,53061,1,0
2020-01-22T00:00:00.000+0000,Snohomish,Washington,53061,1,0
2020-01-23T00:00:00.000+0000,Snohomish,Washington,53061,1,0
2020-01-24T00:00:00.000+0000,Cook,Illinois,17031,1,0
2020-01-24T00:00:00.000+0000,Snohomish,Washington,53061,1,0
2020-01-25T00:00:00.000+0000,Orange,California,6059,1,0
2020-01-25T00:00:00.000+0000,Cook,Illinois,17031,1,0
2020-01-25T00:00:00.000+0000,Snohomish,Washington,53061,1,0
2020-01-26T00:00:00.000+0000,Maricopa,Arizona,4013,1,0
2020-01-26T00:00:00.000+0000,Los Angeles,California,6037,1,0


In [22]:
%sql

SELECT * 
FROM covid 
WHERE county = "Los Angeles"

-- keys = date, grouping = county, values = cases, deaths

date,county,state,fips,cases,deaths
2020-01-26T00:00:00.000+0000,Los Angeles,California,6037,1,0
2020-01-27T00:00:00.000+0000,Los Angeles,California,6037,1,0
2020-01-28T00:00:00.000+0000,Los Angeles,California,6037,1,0
2020-01-29T00:00:00.000+0000,Los Angeles,California,6037,1,0
2020-01-30T00:00:00.000+0000,Los Angeles,California,6037,1,0
2020-01-31T00:00:00.000+0000,Los Angeles,California,6037,1,0
2020-02-01T00:00:00.000+0000,Los Angeles,California,6037,1,0
2020-02-02T00:00:00.000+0000,Los Angeles,California,6037,1,0
2020-02-03T00:00:00.000+0000,Los Angeles,California,6037,1,0
2020-02-04T00:00:00.000+0000,Los Angeles,California,6037,1,0


In [23]:
%sql

SELECT max(cases) AS max_cases, max(deaths) AS max_deaths, county 
FROM covid 
GROUP BY county 
ORDER BY max_cases DESC
LIMIT 10

max_cases,max_deaths,county
155124,11419,New York City
33798,1917,Nassau
31368,1085,Suffolk
29058,1267,Cook
27230,1022,Westchester
19107,895,Los Angeles
15548,1560,Wayne
14738,954,Bergen
13367,655,Hudson
12520,1019,Essex


###Try your own analysis!
* Here's an idea to get you started
* There's a lot more examples [here](https://databricks.com/blog/2020/04/14/covid-19-datasets-now-available-on-databricks.html)

**This is census data taken from census.gov**
* It has enough information to be able to construct a fips code column that will correspond the the NYT data

In [26]:
%sh wget https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/totals/co-est2019-alldata.csv && cp co-est2019-alldata.csv /dbfs/tmp

--2020-04-28 18:27:14-- https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/totals/co-est2019-alldata.csv
Resolving www2.census.gov (www2.census.gov)... 23.14.164.64, 2600:1409:d000:5a5::208c, 2600:1409:d000:5a9::208c
Connecting to www2.census.gov (www2.census.gov)|23.14.164.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘co-est2019-alldata.csv’

 0K .......... .......... .......... .......... .......... 5.16M
 50K .......... .......... .......... .......... .......... 4.06M
 100K .......... .......... .......... .......... .......... 5.09M
 150K .......... .......... .......... .......... .......... 52.2M
 200K .......... .......... .......... .......... .......... 17.9M
 250K .......... .......... .......... .......... .......... 538K
 300K .......... .......... .......... .......... .......... 43.9M
 350K .......... .......... .......... .......... .......... 27.0M
 400K .......... .......... .......... .......... .......... 79.2M
 450K .......... .......... .......... .......... .......... 10.1M
 500K .......... .......... .......... .......... .......... 79.8M
 550K .......... .......... .......... .......... .......... 42.4M
 600K .......... .......... .......... .......... .......... 52.4M
 650K .......... .......... .......... .......... .......... 511K
 700K .......... .......... .......... .......... .......... 20.9M
 750K .......... .......... .......... .......... .......... 10.9M
 800K .......... .......... .......... .......... .......... 31.9M
 850K .......... .......... .......... .......... .......... 117M
 900K .......... .......... .......... .......... .......... 92.5M
 950K .......... .......... .......... .......... .......... 55.5M
 1000K .......... .......... .......... .......... .......... 61.6M
 1050K .......... .......... .......... .......... .......... 83.6M
 1100K .......... .......... .......... .......... .......... 86.8M
 1150K .......... .......... .......... .......... .......... 73.9M
 1200K .......... .......... .......... .......... .......... 79.6M
 1250K .......... .......... .......... .......... .......... 68.7M
 1300K .......... .......... .......... .......... .......... 88.8M
 1350K .......... .......... .......... .......... .......... 50.3M
 1400K .......... .......... .......... .......... .......... 49.2M
 1450K .......... .......... .......... .......... .......... 102M
 1500K .......... .......... .......... .......... .......... 60.2M
 1550K .......... .......... .......... .......... .......... 25.3M
 1600K .......... .......... .......... .......... .......... 24.1M
 1650K .......... .......... .......... .......... .......... 81.8M
 1700K .......... .......... .......... .......... .......... 80.5M
 1750K .......... .......... .......... .......... .......... 94.9M
 1800K .......... .......... .......... .......... .......... 80.8M
 1850K .......... .......... .......... .......... .......... 90.6M
 1900K .......... .......... .......... .......... .......... 39.3M
 1950K .......... .......... .......... .......... .......... 82.3M
 2000K .......... .......... .......... .......... .......... 89.7M
 2050K .......... .......... .......... .......... .......... 75.5M
 2100K .......... .......... .......... .......... .......... 37.2M
 2150K .......... .......... .......... .......... .......... 87.9M
 2200K .......... .......... .......... .......... .......... 79.7M
 2250K .......... .......... .......... .......... .......... 85.6M
 2300K .......... .......... .......... .......... .......... 64.6M
 2350K .......... .......... .......... .......... .......... 94.7M
 2400K .......... .......... .......... .......... .......... 98.2M
 2450K .......... .......... .......... .......... .......... 71.7M
 2500K .......... .......... .......... .......... .......... 82.5M
 2550K .......... .......... .......... .......... .......... 15.3M
 2600K .......... .......... .......... .......... ...

In [27]:
census_df = spark.read.csv("dbfs:/tmp/co-est2019-alldata.csv", header=True, inferSchema=True)

#display() is a Databricks only function. It displays the data, like show(), but also gives the visualization options we saw in the SQL section above
display(census_df)

SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017,POPESTIMATE2018,POPESTIMATE2019,NPOPCHG_2010,NPOPCHG_2011,NPOPCHG_2012,NPOPCHG_2013,NPOPCHG_2014,NPOPCHG_2015,NPOPCHG_2016,NPOPCHG_2017,NPOPCHG_2018,NPOPCHG_2019,BIRTHS2010,BIRTHS2011,BIRTHS2012,BIRTHS2013,BIRTHS2014,BIRTHS2015,BIRTHS2016,BIRTHS2017,BIRTHS2018,BIRTHS2019,DEATHS2010,DEATHS2011,DEATHS2012,DEATHS2013,DEATHS2014,DEATHS2015,DEATHS2016,DEATHS2017,DEATHS2018,DEATHS2019,NATURALINC2010,NATURALINC2011,NATURALINC2012,NATURALINC2013,NATURALINC2014,NATURALINC2015,NATURALINC2016,NATURALINC2017,NATURALINC2018,NATURALINC2019,INTERNATIONALMIG2010,INTERNATIONALMIG2011,INTERNATIONALMIG2012,INTERNATIONALMIG2013,INTERNATIONALMIG2014,INTERNATIONALMIG2015,INTERNATIONALMIG2016,INTERNATIONALMIG2017,INTERNATIONALMIG2018,INTERNATIONALMIG2019,DOMESTICMIG2010,DOMESTICMIG2011,DOMESTICMIG2012,DOMESTICMIG2013,DOMESTICMIG2014,DOMESTICMIG2015,DOMESTICMIG2016,DOMESTICMIG2017,DOMESTICMIG2018,DOMESTICMIG2019,NETMIG2010,NETMIG2011,NETMIG2012,NETMIG2013,NETMIG2014,NETMIG2015,NETMIG2016,NETMIG2017,NETMIG2018,NETMIG2019,RESIDUAL2010,RESIDUAL2011,RESIDUAL2012,RESIDUAL2013,RESIDUAL2014,RESIDUAL2015,RESIDUAL2016,RESIDUAL2017,RESIDUAL2018,RESIDUAL2019,GQESTIMATESBASE2010,GQESTIMATES2010,GQESTIMATES2011,GQESTIMATES2012,GQESTIMATES2013,GQESTIMATES2014,GQESTIMATES2015,GQESTIMATES2016,GQESTIMATES2017,GQESTIMATES2018,GQESTIMATES2019,RBIRTH2011,RBIRTH2012,RBIRTH2013,RBIRTH2014,RBIRTH2015,RBIRTH2016,RBIRTH2017,RBIRTH2018,RBIRTH2019,RDEATH2011,RDEATH2012,RDEATH2013,RDEATH2014,RDEATH2015,RDEATH2016,RDEATH2017,RDEATH2018,RDEATH2019,RNATURALINC2011,RNATURALINC2012,RNATURALINC2013,RNATURALINC2014,RNATURALINC2015,RNATURALINC2016,RNATURALINC2017,RNATURALINC2018,RNATURALINC2019,RINTERNATIONALMIG2011,RINTERNATIONALMIG2012,RINTERNATIONALMIG2013,RINTERNATIONALMIG2014,RINTERNATIONALMIG2015,RINTERNATIONALMIG2016,RINTERNATIONALMIG2017,RINTERNATIONALMIG2018,RINTERNATIONALMIG2019,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RDOMESTICMIG2016,RDOMESTICMIG2017,RDOMESTICMIG2018,RDOMESTICMIG2019,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,RNETMIG2016,RNETMIG2017,RNETMIG2018,RNETMIG2019
40,3,6,1,0,Alabama,Alabama,4779736,4780125,4785437,4799069,4815588,4830081,4841799,4852347,4863525,4874486,4887681,4903185,5312,13632,16519,14493,11718,10548,11178,10961,13195,15504,14226,59690,59067,57929,58903,59647,59389,58961,58271,57313,11075,48833,48366,50851,49712,51876,51710,53195,53665,53879,3151,10857,10701,7078,9191,7771,7679,5766,4606,3434,924,4665,5817,5046,3684,4580,5777,3011,3379,2772,1244,-1893,-114,2297,-959,-1544,-2157,2298,5279,9387,2168,2772,5703,7343,2725,3036,3620,5309,8658,12159,-7,3,115,72,-198,-259,-121,-114,-69,-89,116185,116246,115180,115793,116932,119032,119972,118619,117094,116576,116625,12.455519356,12.286865772,12.011401179,12.180258647,12.305777115,12.225150764,12.109454384,11.938128082,11.707442426,10.189987883,10.060889328,10.543799502,10.279697432,10.702541513,10.644438296,10.925228982,10.994485138,11.005972301,2.2655314734,2.2259764441,1.467601677,1.9005612146,1.6032356022,1.5807124672,1.1842254029,0.9436429432,0.7014701253,0.9734461014,1.2100275652,1.0462726847,0.7617960521,0.944900149,1.1891881655,0.6184014374,0.6922643302,0.5662420464,-0.395012534,-0.023713794,0.4762759328,-0.198306844,-0.318542758,-0.44401573,0.4719649629,1.0815221661,1.9175014754,0.5784335677,1.1863137707,1.5225486174,0.5634892079,0.6263573914,0.7451724354,1.0903664003,1.7737864964,2.4837435218
50,3,6,1,1,Alabama,Autauga County,54571,54597,54773,55227,54954,54727,54893,54864,55243,55390,55533,55869,176,454,-273,-227,166,-29,379,147,143,336,150,638,615,571,640,651,666,676,631,624,157,514,560,582,573,584,547,573,518,541,-7,124,55,-11,67,67,119,103,113,83,25,4,-14,12,7,13,-3,-12,-7,-16,147,327,-329,-226,101,-107,266,59,37,270,172,331,

Let's tweak the DataFrame above to have a fips column that matches the NYT data. Here's the documentation on [user-defined functions (UDFs)](https://docs.databricks.com/spark/latest/spark-sql/udf-python.html).

In [29]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def make_fips(state_code, county_code):
  if len(str(county_code)) == 1:
    return str(state_code) + "00" + str(county_code)
  elif len(str(county_code)) == 2:
    return str(state_code) + "0" + str(county_code)
  else:
    return str(state_code) + str(county_code)

make_fips_udf = udf(make_fips, StringType())
  
census_df = census_df.withColumn("fips", make_fips_udf(census_df.STATE, census_df.COUNTY))

Now that both the census and the covid data have an identical column, let's join the two DataFrames.

In [31]:
covid_with_census = (covid_df
                     .na.drop(subset=["fips"])
                     .join(census_df.drop("COUNTY", "STATE"), on=['fips'], how='inner'))

What do the cases look like for the most populous counties?

In [33]:
display(covid_with_census.filter("POPESTIMATE2019 > 2000000").select("county", "cases", "date"))

# keys = date, grouping = county, values = cases

county,cases,date
Cook,1,2020-01-24T00:00:00.000+0000
Orange,1,2020-01-25T00:00:00.000+0000
Cook,1,2020-01-25T00:00:00.000+0000
Maricopa,1,2020-01-26T00:00:00.000+0000
Los Angeles,1,2020-01-26T00:00:00.000+0000
Orange,1,2020-01-26T00:00:00.000+0000
Cook,1,2020-01-26T00:00:00.000+0000
Maricopa,1,2020-01-27T00:00:00.000+0000
Los Angeles,1,2020-01-27T00:00:00.000+0000
Orange,1,2020-01-27T00:00:00.000+0000


Since the NYT dataset has a new row for every day, with cases increasing each day, let's grab only the most recent numbers for each county.
* Below we're using the `col` function to refer to columns. It's equivalent to something like `df["column_name"]`
* To get the most recent row per county,  we'll use a [window function](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=window#pyspark.sql.Window)

In [35]:
from pyspark.sql.functions import row_number, col
from pyspark.sql import Window

w = Window.partitionBy("fips").orderBy(col("date").desc())
current_covid_rates = (covid_with_census
                       .withColumn("row_num", row_number().over(w))
                       .filter(col("row_num") == 1)
                       .drop("row_num"))

What counties are hardest hit when the cases are scaled with their population?

In [37]:
current_covid_rates = (current_covid_rates
                       .withColumn("case_rates_percent", 100*(col("cases")/col("POPESTIMATE2019")))
                       .sort(col("case_rates_percent").desc()))

#Look at the top 10 counties
display(current_covid_rates.select("county", "state", "cases", "POPESTIMATE2019", "case_rates_percent").limit(10))

county,state,cases,POPESTIMATE2019,case_rates_percent
Lincoln,Arkansas,721,13024,5.535933660933661
Bledsoe,Tennessee,586,15064,3.8900690387679235
Rockland,New York,11091,325789,3.404350668684333
Marion,Ohio,2172,65093,3.336764321816478
Pickaway,Ohio,1662,58457,2.8431154523838034
Westchester,New York,27230,967506,2.8144528302666854
Nassau,New York,33798,1356924,2.490780618516586
Louisa,Iowa,262,11035,2.374263706388763
Dakota,Nebraska,462,20026,2.307000898831519
Randolph,Georgia,153,6778,2.257303039244615
